In [ ]:
import cv2
import numpy as np
import tempfile
import multiprocessing
from sklearn.neighbors import NearestNeighbors
from io_minio import get_image_minio, upload_img_path, upload_parquet
from libs.knn_process import knn_process_df_image
import pyarrow.dataset as ds
import gc
from db_common import insert_data, select_data
from common import generate_hash
import io
from multiprocessing import cpu_count
import boto3
import datetime
from concurrent.futures import ProcessPoolExecutor, as_completed, ThreadPoolExecutor
from tqdm import tqdm
import random
from multiprocessing import Pool, cpu_count
import pyarrow as pa
import pyarrow.parquet as pq
import pandas as pd
import os
import pandas as pd
import numpy as np
import cv2


In [2]:
path = '../dataset'

def get_init_id(table):
    id = select_data(f'SELECT MAX(ID_{table}) i FROM {table}')['i'].iloc[0]
    return 0 if id == None else id

id_product = get_init_id('PRODUCT')
id_data = get_init_id('DATA')

list_product = []
list_objs_data = []

for name in os.listdir(path):
    full_path = os.path.join(path, name)
    if os.path.isdir(full_path):
        id_product = id_product + 1
        
        list_product.append({
            'id_product': id_product,
            'nm_product': name,
            'vl_product': round(5 + (random.random() * 15), 2)
        })
        
        for full_path_file in os.listdir(full_path):
            full_path_file = os.path.join(full_path, full_path_file)
            path_data = generate_hash(full_path_file)
            id_data = id_data + 1
            
            list_objs_data.append({
                'full_path_file': full_path_file,
                'id_product': id_product,
                'id_data': id_data,
                'path_data': path_data
            })

insert_data('product', list_product)
del name, full_path, id_product, id_data, list_product, full_path_file, path, path_data

In [ ]:
datetime_now = datetime.datetime.now()
pc_threads = 70

def process_data(obj_data):
    full_path_file = obj_data['full_path_file']
    id_product = obj_data['id_product']
    id_data = obj_data['id_data']
    path_data = obj_data['path_data']
    
    upload_img_path(full_path_file, key=path_data)
    
    data = {
        'id_data': id_data,
        'path_data': path_data,
        'tp_data': 'IMG', 
        'dt_inclusion': datetime_now
    }
    
    product_data = {
        'id_product': id_product,
        'id_data': id_data
    }
    
    return {'full_path_file':full_path_file, 'path_data':path_data}, data, product_data

list_data = []
list_data_image = []
list_product_data = []

with ProcessPoolExecutor(max_workers=max(1, int(cpu_count() * (pc_threads / 100)))) as executor:
    futures = [executor.submit(process_data, obj) for obj in list_objs_data]

    for future in as_completed(futures):
        data_image, data, product_data = future.result()
        list_data.append(data)
        list_data_image.append(data_image)
        list_product_data.append(product_data)

insert_data('data', list_data)
insert_data('product_data', list_product_data)
del list_data, list_product_data, product_data, list_objs_data, futures, data, datetime_now, pc_threads

d9935bf29be5882ac08df4f1abdef049ed124c1c033ecd6f1ea32f48888591c74fcf466939ebf0ba6bbb90cf504dd868a87ec0c358bb2ed6266457adce7d380942a2f58ae48b5fadf4fd2788387e45f8799d835cd48423ea558d7a787e498e447b6d0bd09a69cbfcca23fa15755e62450fb099acbd20b66ddd426b1d92f811f9121d63363678912e677










ed319fee43462bf7e1c469283
7f43934acfd9b3e40379f04c0c7c6849b77557f206111006d495e83fe9971f81cab7ad2962b0315fc1cadfa61b54f667353aeb6959dd24014b9382d6bad4b484e919f273021df74e8d205fbc7ab606bf993534c951b2d88c7ca14f4fe7d4a4f660ac35fac5c33df57c97a41ca1236ca863c12576840d5747e4913e24b68b2792cb72be443910512abfb










8f1b4cef86ef4e63ef6e081e97b491fbe7abf7587e8b4962a7f6d167cdf423ef14b5f4de8773733a73d46c505873b208c86afba6da316967fae0cb96b12733bfc4c56de499de00bbbc0af4950f6e5e7d4f082990d9c164375b8492b1743bfbcf38c79293e874d3da9bde5f49d87be22e8








87fc4001447af699dfd32797b58284e643533fa9b3805d43da

e039bdf90e6046fe6aa88ed1662b0a0cb27fbe55453497792a76432a44f6ff8756066e2c01c01d7147557317f23f13ee387f4070ed4960dd6ee6

In [ ]:
def process(path_data):
    np_array = np.frombuffer(get_image_minio(path_data), np.uint8)
    img_original = cv2.imdecode(np_array, cv2.IMREAD_COLOR)
    return pd.DataFrame([knn_process_df_image(image_process=img_original)])

df_resultado = select_data('SELECT pd.ID_PRODUCT, pd.ID_PRODUCT_DATA, d.PATH_DATA FROM PRODUCT_DATA pd JOIN DATA d ON d.ID_DATA = pd.ID_DATA')

processed_list = []
for _, row in df_resultado.iterrows():
    try:
        df_proc = process(row["path_data"])
        df_proc["id_product"] = row["id_product"]
        df_proc["id_product_data"] = row["id_product_data"]
        df_proc["path_data"] = row["path_data"]
        processed_list.append(df_proc)
    except Exception as e:
        print(f"Erro em {row['path_data']}: {e}")

df_result_final = pd.concat(processed_list, ignore_index=True)
df_result_final.to_parquet('df_result_final.parquet')

,image_process,img_rgb,img_cinza,img_suavizada,img_bordas_canny,mascara_segmentada,contornos,altura_img,largura_img,contornos_filtrados,...,img_com_contornos,metricas_geo,vetor_hog,img_visual_hog,img_visual_lbp,hist_lbp,metricas_glcm,id_product,id_product_data,path_data
0,"[255, 255, 255, 255, 255, 255, 255, 255, 255, ...","[255, 255, 255, 255, 255, 255, 255, 255, 255, ...","[255, 255, 255, 255, 255, 255, 255, 255, 255, ...","[255, 255, 255, 255, 255, 255, 255, 255, 255, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[48.0, 11.0, 47.0, 12.0, 41.0, 12.0, 40.0, 13....",[100],[100],[],...,"[255, 255, 255, 255, 255, 255, 255, 255, 255, ...",[],"[0.0017182488668723718, 0.0, 0.0, 0.0, 0.00558...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[84, 141, 141, 141, 141, 141, 141, 141, 141, 1...","[0.0443, 0.0648, 0.0407, 0.0946, 0.1175, 0.156...","[678.5398328717337, 9.654323738024315, 0.35781...",1,1,fb099acbd20b66ddd426b1d92
1,"[255, 255, 255, 255, 255, 255, 255, 255, 255, ...","[255, 255, 255, 255, 255, 255, 255, 255, 255, ...","[255, 255, 255, 255, 255, 255, 255, 255, 255, ...","[255, 255, 255, 255, 255, 255, 255, 255, 255, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[53.0, 0.0, 53.0, 1.0, 52.0, 2.0, 51.0, 2.0, 5...",[100],[100],[],...,"[255, 255, 255, 255, 255, 255, 255, 255, 255, ...",[],"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[84, 141, 141, 141, 141, 141, 141, 141, 141, 1...","[0.0527, 0.0665, 0.0417, 0.0713, 0.0861, 0.134...","[613.1155461480909, 9.231754312615475, 0.40244...",1,2,c0c358bb2ed6266457adce7d3
2,"[255, 255, 255, 255, 255, 255, 255, 255, 255, ...","[255, 255, 255, 255, 255, 255, 255, 255, 255, ...","[255, 255, 255, 255, 255, 255, 255, 255, 255, ...","[255, 255, 255, 255, 255, 255, 255, 255, 255, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[37.0, 14.0, 36.0, 15.0, 34.0, 15.0, 33.0, 16....",[100],[100],[],...,"[255, 255, 255, 255, 255, 255, 255, 255, 255, ...",[],"[0.0, 0.0, 0.0, 0.0, 0.003836284540312295, 0.0...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[84, 141, 141, 141, 141, 141, 141, 141, 141, 1...","[0.0452, 0.0634, 0.041, 0.076, 0.0901, 0.1374,...","[682.0651179891457, 9.750183513978254, 0.40857...",1,3,d9935bf29be5882ac08df4f1a
3,"[255, 255, 255, 255, 255, 255, 255, 255, 255, ...","[255, 255, 255, 255, 255, 255, 255, 255, 255, ...","[255, 255, 255, 255, 255, 255, 255, 255, 255, ...","[255, 255, 255, 255, 255, 255, 255, 255, 255, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[44.0, 12.0, 43.0, 13.0, 38.0, 13.0, 37.0, 14....",[100],[100],[],...,"[255, 255, 255, 255, 255, 255, 255, 255, 255, ...",[],"[0.0, 0.0, 0.0, 0.0, 0.006137725613211971, 0.0...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[84, 141, 141, 141, 141, 141, 141, 141, 141, 1...","[0.0416, 0.0653, 0.0404, 0.09, 0.119, 0.153, 0...","[712.7515410278857, 9.786879546098985, 0.36474...",1,4,f811f9121d63363678912e677
4,"[255, 255, 255, 255, 255, 255, 255, 255, 255, ...","[255, 255, 255, 255, 255, 255, 255, 255, 255, ...","[255, 255, 255, 255, 255, 255, 255, 255, 255, ...","[255, 255, 255, 255, 255, 255, 255, 255, 255, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[32.0, 2.0, 30.0, 4.0, 29.0, 4.0, 29.0, 5.0, 2...",[100],[100],[],...,"[255, 255, 255, 255, 255, 255, 255, 255, 255, ...",[],"[0.00649006611659874, 0.0, 0.0, 0.0, 0.0032450...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[84, 141, 141, 141, 141, 141, 141, 141, 141, 1...","[0.0371, 0.0651, 0.0431, 0.1031, 0.1267, 0.156...","[622.9834524264448, 9.566012555758109, 0.34995...",1,5,88387e45f8799d835cd48423e
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...

In [ ]:
from io_minio import upload_parquet
import pandas as pd

df_result_final = pd.read_parquet('df_result_final.parquet')

for idx, row in df_result_final.iterrows():
    upload_parquet(pd.DataFrame([row]), row['path_data'])